In [2]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import time
import serial

In [3]:
key = "nyVvF841UBPdCn%2BvugXzKE1EEwkhTBIhjhaXOQgC%2FAlD2F0YongS3tfgXgIxbW5lmlkuJ6Szjamn18IPVNMpXA%3D%3D"

In [4]:
# 아두이노 시리얼 포트와 속도 설정
serial_port = "/dev/tty.usbserial-0001"
baud_rate = 115200

# 시리얼 포트 연결
ard = serial.Serial(serial_port, baud_rate)

In [5]:
def getArrivalInfo(arsId):
    # 정류소고유번호(arsId)를 이용하여 해당 정류소의 정보를 가져옴
    html = requests.get('http://ws.bus.go.kr/api/rest/stationinfo/getStationByUid?serviceKey='+key+'&arsId='+arsId).text
    root = BeautifulSoup(html, 'xml')
    # 정류소명과 버스도착정보목록을 출력
    station_name = root.find('stNm').text
    # 정류소에 도착하는 버스 정보를 가져옴
    items = root.find_all('itemList')
    
    # 버스 정보를 저장할 문자열 변수
    bus_info_str = ""
    
    for item in items:
        # 버스번호, 남은시간 등의 정보를 출력
        bus_route_id = item.find('busRouteId').text
        bus_route_name = item.find('rtNm').text
        predict_msg = item.find('arrmsg1').text  # 예상 도착 메시지
        if predict_msg == '운행종료':
            bus_info_str += f'{bus_route_name}: 운행종료!\n'
        elif predict_msg == '곧 도착':
            bus_info_str += f'{bus_route_name}: 곧 도착!\n'
        else:
            # 예상 도착 시간과 "후"를 분리하여 처리
            time, suffix = predict_msg.split('후')
            time = time.strip()  # 앞뒤 공백 제거
            # 예상 도착 시간을 '분'과 '초'로 분리하여 출력
            minutes, seconds = time.split('분')
            bus_info_str += f'{bus_route_name}: {minutes.strip()} 분 {seconds} 후!\n'
    
    # 모든 버스 정보를 저장한 문자열 반환
    return bus_info_str


In [6]:
# 4개의 정류소에 대해 1분마다 정보를 출력하는 함수
def main():
    arsIds = ['05251']  # 4개의 정류소고유번호
    status = 0
    while True:
        for arsId in arsIds:
            rst = getArrivalInfo(arsId)
            print(rst)
            ard.write(rst.encode())  # 문자열을 바이트로 인코딩하여 시리얼 포트로 전송
            # ard.write(rst)
        time.sleep(10)  # 1분 간격으로 API 값을 불러옴

In [7]:
if __name__ == "__main__":
    main()

422: 3 분 55초 후!
721: 14 분 10초 후!
N61: 운행종료!
N62: 운행종료!
2227: 6 분 40초 후!
2416: 6 분 59초 후!
3216: 7 분 18초 후!
4212: 8 분 57초 후!
3500포천: 9 분 37초 후!
680구리: 24 분 32초 후!

422: 3 분 39초 후!
721: 14 분 1초 후!
N61: 운행종료!
N62: 운행종료!
2227: 6 분 26초 후!
2416: 6 분 46초 후!
3216: 7 분 9초 후!
4212: 8 분 23초 후!
3500포천: 9 분 37초 후!
680구리: 24 분 18초 후!

422: 3 분 27초 후!
721: 14 분 6초 후!
N61: 운행종료!
N62: 운행종료!
2227: 6 분 16초 후!
2416: 6 분 38초 후!
3216: 7 분 9초 후!
4212: 8 분 18초 후!
3500포천: 8 분 49초 후!
680구리: 24 분 18초 후!

422: 2 분 39초 후!
721: 13 분 46초 후!
N61: 운행종료!
N62: 운행종료!
2227: 6 분 16초 후!
2416: 6 분 27초 후!
3216: 6 분 50초 후!
4212: 8 분 4초 후!
3500포천: 8 분 42초 후!
680구리: 23 분 57초 후!

422: 2 분 39초 후!
721: 13 분 40초 후!
N61: 운행종료!
N62: 운행종료!
2227: 5 분 54초 후!
2416: 6 분 15초 후!
3216: 6 분 50초 후!
4212: 7 분 43초 후!
3500포천: 8 분 42초 후!
680구리: 23 분 57초 후!

422: 2 분 39초 후!
721: 13 분 26초 후!
N61: 운행종료!
N62: 운행종료!
2227: 5 분 54초 후!
2416: 6 분 15초 후!
3216: 6 분 30초 후!
4212: 7 분 43초 후!
3500포천: 8 분 18초 후!
680구리: 23 분 21초 후!

422: 2 분 13초 후!
721: 13 분 17초 후!


KeyboardInterrupt: 